In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Sudoku Solver using Annealing

Napisz program poszukujący rozwiązania łamigłówki Sudoku za pomocą symulowanego
wyżarzania. Plansza 9 ×9 ma zostać wczytana z pliku tekstowego, w którym pola puste
zaznaczone są znakiem x. Jako funkcję kosztu przyjmij sumę powtórzeń cyfr występu-
jących w wierszach bloku 9 ×9, kolumnach bloku 9 ×9 oraz blokach 3 ×3. Zaproponuj
metodę generacji stanu sąsiedniego. Przedstaw zależność liczby iteracji algorytmu od
liczby pustych miejsc na planszy. Czy Twój program jest w stanie znaleźć poprawne
rozwiązanie dla każdej z testowanych konfiguracji wejściowych?

## Table of Contents

1. [Read Sudokus](#1-read-sudokus)
2. [Annealing](#2-annealing)  
    2.1 [Cost Function](#21-cost-function)  
    2.2 [Neighbourhood](#22-neighbourhood)  
    2.3 [Annealing](#23-annealing)

## 1. Read Sudokus

sudokus were taken from [http://magictour.free.fr/top1465](http://magictour.free.fr/top1465)

In [27]:
def read_sudokus(filename="sudoku.txt"):
    with open(filename, "r") as f:
        sudokus = f.readlines()
    sudokus = [list(sudoku.strip()) for sudoku in sudokus]
    return np.array([np.reshape(row, (-1, 9, 9)) for row in sudokus])

## 2. Annealing

### 2.1 Cost Function

### 2.2 Neighbourhood

### 2.3 Annealing